In [ ]:
# !wget https://www.cs.ccu.edu.tw/~wtchu/projects/Weather/Image.zip
# !unzip Image.zip
# !mv Image weather_dataset

# Uncomment to run on Colab 

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
# !pip install pyunpack
# !pip install patool
# from pyunpack import Archive

# Archive('/content/gdrive/My Drive/Colab Notebooks/Project/Image.rar').extractall('./')

# Split Dataset to Train, Val, Test

In [ ]:
!pip install split-folders

In [ ]:
import split_folders

In [ ]:
split_folders.ratio("Image", output="weather_dataset", seed=1337, ratio=(0.7,0.15,0.15))

In [ ]:
# Uncomment to run on colab, Change paths.

# !cp gdrive/My\ Drive/Colab\ Notebooks/Project/*.sh  .
# !cp gdrive/My\ Drive/Colab\ Notebooks/Project/*.py  .
# !chmod +x augment_images.sh
# !./augment_images.sh "./weather_dataset/train/"

In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split as rsplit
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os
import torchvision.models
import meta_parser

torch.manual_seed(1) 
np.random.seed(1)

In [2]:
def convert_to_dataset(mdata, dataset):
    for ids, weather_feats in mdata.items():
#         print(weather_feats[0])
        feats = np.asarray([float(list(wfeat.values())[0]) for wfeat in weather_feats[0]],dtype=np.float64)
        label = np.asarray(float(classes.index(weather_feats[1])),dtype=np.float64)
        print(feats)
#         print(type(feats), feats)
#         print(type(label), label)
        dataset.append([feats, label])
#         print(dataset)
    

In [3]:
m = meta_parser.metadata_map()

imgs_ids * 4 is equal to set(img_ids):  True
Number of Image Ids:  5883
Number of Image Ids with metadata:  4569
imgs_ids * 4 is equal to set(img_ids):  False
Number of Image Ids:  1258
Number of Image Ids with metadata:  969
imgs_ids * 4 is equal to set(img_ids):  False
Number of Image Ids:  1267
Number of Image Ids with metadata:  971
3


In [4]:
train_mdata= m.get_available_metadata("train")
val_mdata = m.get_available_metadata("val")
test_mdata = m.get_available_metadata("test")

In [6]:
print(type(train_mdata))

classes = ["cloudy", "foggy", "rain", "snow", "sunny"]

train_data = []
val_data = []
test_data = []

convert_to_dataset(train_mdata, train_data)
convert_to_dataset(val_mdata, val_data)
convert_to_dataset(test_mdata, test_data)



<class 'dict'>
[ 5.700e+00  8.100e+01  7.000e+00  1.033e+03  4.000e+00 -9.999e+03
  1.000e+01]
[ 5.000e-01  8.100e+01  2.000e+00  1.029e+03 -1.000e+00 -9.999e+03
  1.000e+01]
[-3.600e+00  8.100e+01  2.000e+00  1.010e+03 -1.000e+00 -9.999e+03
  5.000e+00]
[-6.800e+00  9.300e+01 -2.000e+00  1.020e+03 -3.000e+00 -9.999e+03
  9.000e+00]
[-6.200e+00  9.300e+01 -1.000e+00  1.001e+03 -2.000e+00 -9.999e+03
  1.000e+01]
[-7.500e+00  8.000e+01 -5.000e+00  1.035e+03 -8.000e+00 -9.999e+03
  8.000e+00]
[-9.990e+02  8.300e+01  2.500e+01  1.018e+03  2.200e+01 -9.999e+03
  7.000e+00]
[-9.990e+02  6.800e+01  2.000e+01  1.011e+03  1.400e+01 -9.999e+03
  9.000e+00]
[-7.200e+00  1.000e+02 -2.000e+00  9.930e+02 -2.000e+00 -9.999e+03
  1.000e+01]
[-6.500e+00  8.600e+01 -1.000e+00  1.028e+03 -3.000e+00 -9.999e+03
  1.000e+01]
[-3.700e+00  9.300e+01  1.000e+00  1.025e+03  0.000e+00 -9.999e+03
  1.500e+00]
[-9.990e+02  8.700e+01  1.000e+01  1.014e+03  8.000e+00 -9.999e+03
  8.000e+00]
[-9.990e+02  8.700e+01  9

In [7]:
print(train_data[0])

[array([ 5.700e+00,  8.100e+01,  7.000e+00,  1.033e+03,  4.000e+00,
       -9.999e+03,  1.000e+01]), array(0.)]


In [8]:
trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=5)
i1, l1 = next(iter(trainloader))
print(i1.shape)

torch.Size([5, 7])
